In [1]:
from datasets import load_dataset
dataset = load_dataset("yaful/DeepfakeTextDetect")

Found cached dataset deepfake_text_detect (C:/Users/Rahulram/.cache/huggingface/datasets/yaful___deepfake_text_detect/default/1.0.0/ea38a19203d63b7e8774306c902cc3732161c32360f898bc8a14e09efeeb802b)


  0%|          | 0/5 [00:00<?, ?it/s]

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.DataFrame(dataset["test_ood_gpt"])

In [4]:
llm = df[df["label"] == 0].sample(n=50, random_state=69)
human = df[df["label"] == 1].sample(n=50, random_state=69)
new_df = pd.concat([llm, human])

new_df

,text,label,src
334,Japanese Fish Found in Tsunami Debris Highligh...,0,cnn_gpt4
678,"Person1: Well, the facilities are impressive a...",0,dialogsum_gpt4
1435,Pharmacotherapy for Fibromyalgia: An Update Fi...,0,pubmed_gpt4
218,Bucks Resident Builds Remarkable Observatory i...,0,cnn_gpt4
1554,Oral contraceptives can potentially impact the...,0,pubmed_gpt4
...,...,...,...
131,Adi Viveash's Chelsea U19s side stormed into t...,1,cnn_human
405,"Person1: Where are you going, Jane? Person2: I...",1,dialogsum_human
948,Fires on the plain directed by Kon Ichikawa an...,1,imdb_human
876,Leaving aside the drawbacks and deficiencies o...,1,imdb_human


In [5]:
new_df.to_csv("Sample.csv")

### Base Arch

In [6]:
import torch
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# init
device = 'cpu' # use 'cuda:0' if GPU is available
model_dir = "nealcly/detection-longformer"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)

In [7]:
output = []

for text in new_df.iloc[:, 0]:
    inputs = tokenizer(text, return_tensors="pt")
    logits = model(**inputs).logits
    predicted_class_id = int(torch.argmax(logits, axis=-1)[0])
    output.append(predicted_class_id)

In [8]:
output

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1]

In [9]:
new_df["base"] = output
new_df.to_csv("Base.csv")